### Statistical Analysis from <i>Penetrating Abdominal Trauma with Hollow Viscus Injury: Infectious Risks Revisited<i/>

David T. Pointer Jr, MD, Lili E. Schindelar, BS, Matthew D. Zelhart, MD, Zaid Al-Qurayshi, MD, MPH, Douglas P. Slakey, MD, MPH, FACS, Ronald L. Nichols, MD, MS, FACS

Department of Surgery, Tulane University School of Medicine, New Orleans, LA, USA

**Background:** <i> Surgical site infections (SSI) have a great impact on morbidity and mortality in the surgical patient. The Eastern Association for the Surgery of Trauma has established guidelines for the use of prophylactic antibiotics in penetrating abdominal trauma. We aimed to determine SSI associated risk factors in patients sustaining penetrating abdominal trauma with hollow viscus injury, analyze perioperative antibiotic type and administration, and determine adherence to recommended guidelines.<i/>

In [108]:
import pandas as pd
import numpy as np

#import cleaned data and check the data/column names
df = pd.read_csv("trauma_cleaned.csv")
#df.describe()
#make this more intuitive
df['survival'] = df.died == 1
df.keys()

Index(['id', 'mrn', 'act_no', 'admit_date', 'age', 'gender', 'wt', 'race',
       'inj_type', 'inj_site', 'sbp', 'dbp', 'rr', 'pr', 'o2', 'gcs', 'shock',
       'dispos', 'proc_date', 'time', 'inj_org', 'Bld_unit', 'ostomy', 'wound',
       'antibio', 'dose1', 'day1', 'day2', 'day_beyond', 'ssi', 'ssi_org',
       'absc', 'absc_org', 'pneu', 'septic', 'sept_org', 'cdiff', 'inf2',
       'died', 'comp', 'los', 'infprop', 'survprop', 'survival'],
      dtype='object')

### VAC Patients
Was there a significant difference between group 3 and groups 1 and 2 for the following:
* SSI (ssi = 'Y')
* Abscess (absc = 'Y')
* Sepsis (septic = 'Y')
* c diff infection (cdiff = 'Y')
* Survival (survival = True)
* Days in Hospital (sol is a continuous variable, use student's T-test)


In [109]:
from scipy.stats import fisher_exact
from scipy.stats import ttest_ind

#VAC patients from wound = 3
#VAC vs SSI
table = pd.crosstab(df.ssi =='Y',df.wound == 3)
print("VAC (wound = True) vs SSI (True)")
print(table)
print("p-value: ", fisher_exact(table)[1])
print("____________________")

#VAC vs Abscess
print("VAC (wound = True) vs Abscess (True)")
table = pd.crosstab(df.absc =='Y',df.wound == 3)
print(table)
print("p-value: ", fisher_exact(table)[1])
print("____________________")

#VAC vs Sepsis
print("VAC (wound = True) vs Sepsis (True)")
table = pd.crosstab(df.septic =='Y',df.wound == 3)
print(table)
print("p-value: ", fisher_exact(table)[1])
print("____________________")

#VAC vs c diff Infection
print("VAC (wound = True) vs c diff Infection (True)")
table = pd.crosstab(df.cdiff =='Y',df.wound == 3)
print(table)
print("p-value: ", fisher_exact(table)[1])
print("____________________")

#VAC vs Survival
print("VAC (wound = True) vs Survival (True)")
table = pd.crosstab(df.survival ==True,df.wound == 3)
print(table)
print("p-value: ", fisher_exact(table)[1])
print("____________________")


#VAC vs Days in Hospital
vac = df[df['wound']==3]
non_vac = df[df['wound']!=3]
print("Average Days in the Hospital per Wound Type (VAC = 3)")
print(pd.pivot_table(df, values='los', columns=['wound']))
print("p_value: ",ttest_ind(vac['los'], non_vac['los'])[1])


VAC (wound = True) vs SSI (True)
wound  False  True 
ssi                
False    102     41
True      15     20
p-value:  0.00253025884135
____________________
VAC (wound = True) vs Abscess (True)
wound  False  True 
absc               
False     99     38
True      18     23
p-value:  0.00128251397465
____________________
VAC (wound = True) vs Sepsis (True)
wound   False  True 
septic              
False     110     49
True        7     12
p-value:  0.00892667579243
____________________
VAC (wound = True) vs c diff Infection (True)
wound  False  True 
cdiff              
False    116     57
True       1      4
p-value:  0.047621479644
____________________
VAC (wound = True) vs Survival (True)
wound     False  True 
survival              
False         5      8
True        112     53
p-value:  0.0638045652007
____________________
Average Days in the Hospital per Wound Type (VAC = 3)
wound
1.0    11.043103
2.0    12.000000
3.0    36.737705
Name: los, dtype: float64
p_value:  3.80444095

Wound type is statistically significant in all of the above cases. However, at 95% confidence, we cannot reject the null hypothesis for survival.

### SSI superficial vs Abscess   
Was there a difference between superficial Surgical Site Infections vs Abscess in the following:
*	Survival  (survival = True)
*	Sepsis (septic = 'Y') <br> <br>
Note: Superficial SSI is denoted by ssi = 'Y' AND absc = 'N'; Abscess denoted by absc = 'Y'

In [111]:
def sssi_vs_abscess (row):
    if row['ssi'] == 'Y' and row['absc'] == 'N':
        return 's'
    if row['absc'] == 'Y':
        return 'a'
    return 'other'
df['superficial_ssi'] = df.apply (lambda row: sssi_vs_abscess(row),axis=1)

#make sure that worked
print(df[['ssi','absc','superficial_ssi']].head()) #nice :)

sub_df = df[df.superficial_ssi != 'other']
print("\n")
print("Now we are only looking at " + str(len(sub_df)) + " patients")
print("\n")

#Superficial SSI vs Survival
print("Superficial SSI (s_vs_a = True) vs Survival (True)")
table = pd.crosstab(sub_df.survival ==True,sub_df.superficial_ssi == 's')
print(table)
print("p-value: ", fisher_exact(table)[1])
print("____________________")

#Superficial SSI vs Sepsis
print("Superficial SSI (s_vs_a = True) vs Sepsis (True)")
table = pd.crosstab(sub_df.septic =='Y',sub_df.superficial_ssi == 's')
print(table)
print("p-value: ", fisher_exact(table)[1])
print("______________________")


  ssi absc superficial_ssi
0   Y    Y               a
1   Y    N               s
2   N    N           other
3   N    N           other
4   N    N           other


Now we are only looking at 62 patients


Superficial SSI (s_vs_a = True) vs Survival (True)
superficial_ssi  False  True 
survival                     
False                0      2
True                41     19
p-value:  0.111052353252
____________________
Superficial SSI (s_vs_a = True) vs Sepsis (True)
superficial_ssi  False  True 
septic                       
False               33     15
True                 8      6
p-value:  0.524092004229
______________________


Neither of these tests are statistically significant

### Use of Antibiotics > 1 day vs. 1 day or less 
(Note: if a patient has **any** prescriptions for $2^+$ days then they are incuded in Antibiotics > 1 days)

* SSI (ssi = 'Y')
* Abscess (absc = 'Y')
* Sepsis (septic = 'Y')
* c diff infection (cdiff = 'Y')
* Survival (survival = True)
* Days in Hospital (sol is a continuous variable, use student's T-test)

In [113]:
boolean_df = df.isnull()

def days (row):
    if row['day2'] == False or row['day_beyond'] == False:
        return 2
    if row['dose1'] == False or row['day1'] == False:
        return 1
    return 0
df['days'] = boolean_df.apply (lambda row: days(row),axis=1)

#make sure that worked
print(df[['dose1','day1','day2','day_beyond','days']].head()) #nice :)
#quickly check how many patients in each group
print("\n")
print("Antibiotics >1 days: ",len(df[df.days==2]))
print("Antibiotics 1 day: ",len(df[df.days!=2]))

#4 patients did not receive antibiotics
print("\n")
print("Taking these 4 patients who received no antibiotics out of the analysis:")
print(df[['dose1','day1','day2','day_beyond','days']][df.days==0])
#so take them out of analysis
anti_df = df[df.days!=0]

  dose1     day1 day2 day_beyond  days
0   NaN  Mefoxin  NaN        NaN     1
1   NaN  Mefoxin  NaN      Ancef     2
2   NaN    Ancef  NaN        NaN     1
3   NaN  Mefoxin  NaN        NaN     1
4   NaN    Ancef  NaN    Mefoxin     2


Antibiotics >1 days:  55
Antibiotics 1 day:  123


Taking these 4 patients who received no antibiotics out of the analysis:
    dose1 day1 day2 day_beyond  days
80    NaN  NaN  NaN        NaN     0
106   NaN  NaN  NaN        NaN     0
131   NaN  NaN  NaN        NaN     0
167   NaN  NaN  NaN        NaN     0


In [114]:
#Days vs SSI
table = pd.crosstab(anti_df.ssi =='Y',anti_df.days == 2)
print("Days (days = True means 2+ days) vs SSI (True)")
print(table)
print("p-value: ", fisher_exact(table)[1])
print("____________________")

#Days vs Abscess
print("Days (days = True means 2+ days) vs Abscess (True)")
table = pd.crosstab(anti_df.absc =='Y',anti_df.days == 2)
print(table)
print("p-value: ", fisher_exact(table)[1])
print("____________________")

#Days vs Sepsis
print("Days (days = True means 2+ days) vs Sepsis (True)")
table = pd.crosstab(anti_df.septic =='Y',anti_df.days == 2)
print(table)
print("p-value: ", fisher_exact(table)[1])
print("____________________")

#Days vs c diff Infection
print("Days (days = True means 2+ days) vs c diff Infection (True)")
table = pd.crosstab(anti_df.cdiff =='Y',anti_df.days == 2)
print(table)
print("p-value: ", fisher_exact(table)[1])
print("____________________")

#Days vs Survival
print("Days (days = True means 2+ days) vs Survival (True)")
table = pd.crosstab(anti_df.survival ==True,anti_df.days == 2)
print(table)
print("p-value: ", fisher_exact(table)[1])
print("____________________")


#Days vs Days in Hospital
multiple = anti_df[anti_df['days']==2]
one = anti_df[anti_df['days']==1]
print("Average Days in the Hospital per Group (days = 2+ vs days = 1)")
print(pd.pivot_table(anti_df, values='los', columns=['days']))
print("p_value: ",ttest_ind(vac['los'], non_vac['los'])[1])

Days (days = True means 2+ days) vs SSI (True)
days   False  True 
ssi                
False    101     38
True      18     17
p-value:  0.0242178939665
____________________
Days (days = True means 2+ days) vs Abscess (True)
days   False  True 
absc               
False     89     44
True      30     11
p-value:  0.565128440384
____________________
Days (days = True means 2+ days) vs Sepsis (True)
days    False  True 
septic              
False     111     44
True        8     11
p-value:  0.0163258999652
____________________
Days (days = True means 2+ days) vs c diff Infection (True)
days   False  True 
cdiff              
False    116     53
True       3      2
p-value:  0.651793652703
____________________
Days (days = True means 2+ days) vs Survival (True)
days      False  True 
survival              
False         9      3
True        110     52
p-value:  0.754807000371
____________________
Average Days in the Hospital per Group (days = 2+ vs days = 1)
days
1    16.302521
2    28.5

The two tests here that are significant are Sepsis and SSI (patients with only one dose/day of antibiotics are associated with fewer cases of Sepsis and SSI).


# Recreating Analysis from Superficial SSI for all SSI
1) Search for statistically significant variables following results in tables 3 <br>
2) Perform Logistic regression on these variables (http://blog.yhat.com/posts/logistic-regression-and-python.html)

In [63]:
import pandas as pd
import numpy as np
'''
Prep data

We want days of antibiotics and superficial ssi in the data set

'''
#import cleaned data and check the data/column names
df = pd.read_csv("trauma_cleaned.csv")
print(len(df))

df['survival'] = df.died == 1
print("Keys: ",df.keys())

#Create Superficial SSI Column
def sssi_vs_abscess (row):
    if row['ssi'] == 'Y' and row['absc'] == 'N':
        return 1
    if row['absc'] == 'Y':
        return 0
    return 0
#superficial_ssi = 1 means it was superficial surgical site infection
df['superficial_ssi'] = df.apply (lambda row: sssi_vs_abscess(row),axis=1)

#returns True if entry is empty (so False means we dispensed an antibiotic)
boolean_df = df.isnull()

def days (row):
    if row['day2'] == False or row['day_beyond'] == False:
        return 1
    if row['dose1'] == False or row['day1'] == False:
        return 0
    return 0
df['2_plus_abdays'] = boolean_df.apply (lambda row: days(row),axis=1)


'''
Create other dummies:

gender == 'Female'  (gender)
injury type   (inj_type)
hollow viscus injury (3 way table!) (inj_site)
# injured organs (inj_org)
Pulse         (pr)
# blood units (Bld_unit)
Length of Stay (los)
VAC wound (wound == 3)
Days of antibiotics (days == 2)
'''
df['dummy_ssi'] = df.ssi == 'Y'
df['female'] = df['gender']=='Female'
df['hollow_visc'] = df['inj_site'].str.contains("Colon")
df['gunshot'] = df['inj_type'] == 1
df['3_plus_wounds'] = df['inj_org'] > 2
df['120_plus_pulse'] = df['pr'] >= 120
df['1_plus_blood'] = df['Bld_unit'] >= 1
df['10_plus_days'] = df['los'] > 9
df['vac'] = df['wound'] == 3

178
Keys:  Index(['id', 'mrn', 'act_no', 'admit_date', 'age', 'gender', 'wt', 'race',
       'inj_type', 'inj_site', 'sbp', 'dbp', 'rr', 'pr', 'o2', 'gcs', 'shock',
       'dispos', 'proc_date', 'time', 'inj_org', 'Bld_unit', 'ostomy', 'wound',
       'antibio', 'dose1', 'day1', 'day2', 'day_beyond', 'ssi', 'ssi_org',
       'absc', 'absc_org', 'pneu', 'septic', 'sept_org', 'cdiff', 'inf2',
       'died', 'comp', 'los', 'infprop', 'survprop', 'survival'],
      dtype='object')


In [135]:
from scipy.stats import fisher_exact
from tabulate import tabulate

#----------------------Support functions -------------------------------------#
#loop over this guy to find statistically significant variables
def OR(df, superficial, reference, op, test):
    '''
    Input:
        df - dataframe
        superficial - Boolean (superficial vs. regular)
        reference - column to test
        op - valid operations {lt, gt, eq}
        test - i.e. df.reference > test
    Output:
        table: OR UC LC P_value
    '''
    if op == "lt":
        if superficial:
            table = pd.crosstab(df.superficial_ssi == 1, df[reference] < test)
        else:
            table = pd.crosstab(df.dummy_ssi == 1, df[reference] < test)
    elif op == "gt":
        if superficial:
            table = pd.crosstab(df.superficial_ssi == 1, df[reference] > test)
        else:
            table = pd.crosstab(df.dummy_ssi == 1, df[reference] > test)
    elif op == "eq":
        if superficial:
            table = pd.crosstab(df.superficial_ssi == 1, df[reference] == test)
        else:
            table = pd.crosstab(df.dummy_ssi == 1, df[reference] == test)
    else:
        return "sorry not a valid operation"
    
    #print(table)
    a = table[0][0]
    b = table[0][1]
    c = table[1][0]
    d = table[1][1]
    
    #odds ratio
    test_odds = d/(c)
    ref_odds = b/(a)
    OR = test_odds/ref_odds
    
    #conf intervals
    UC = np.exp(np.log(OR)+1.96*(1/a+1/(b)+1/c+1/(d))**.5)
    LC = np.exp(np.log(OR)-1.96*(1/a+1/(b)+1/c+1/(d))**.5)
    
    p_val = fisher_exact(table)[1]
    print(reference + " " + op + " " + str(test))
    t = tabulate([[OR, UC,LC,p_val]], headers=['OR', 'UC','LC','p_val'])
    return t,p_val
'''
Variables to test:
gender == 'Female'  (gender)
injury type   (inj_type)
hollow viscus injury (3 way table!) (inj_site)
# injured organs (inj_org)
Pulse         (pr)
# blood units (Bld_unit)
Length of Stay (los)
VAC wound (wound == 3)
Days of antibiotics (days == 2)
'''

X_cols = ['2_plus_abdays', 'female',
       'hollow_visc', 'gunshot', '3_plus_wounds', '120_plus_pulse',
       '1_plus_blood', '10_plus_days', 'vac']

#For example, SSI for test days >= 10:
t,p_val = OR(df,False,"los",'gt',9)
print(t) #this matches results our friend got

los gt 9
     OR       UC       LC       p_val
-------  -------  -------  ----------
7.40625  20.1829  2.71778  9.0237e-06


### SSI Results:

In [136]:
# odds ratios and P-Values
sig_vars = []
for var in X_cols:
    t,p_val = OR(df,False,var,'eq',1)
    if p_val < .15:
        sig_vars.append(var)
    print(t)
    print("\n")
print(sig_vars)

2_plus_abdays eq 1
     OR       UC       LC      p_val
-------  -------  -------  ---------
2.60965  5.57824  1.22086  0.0147861


female eq 1
     OR      UC        LC     p_val
-------  ------  --------  --------
1.42222  4.2192  0.479408  0.552268


hollow_visc eq 1
     OR       UC        LC     p_val
-------  -------  --------  --------
1.92344  4.21956  0.876784  0.128434


gunshot eq 1
     OR       UC        LC     p_val
-------  -------  --------  --------
2.52823  11.4078  0.560312  0.378168


3_plus_wounds eq 1
     OR       UC        LC     p_val
-------  -------  --------  --------
1.75667  3.70656  0.832546  0.173007


120_plus_pulse eq 1
     OR       UC       LC     p_val
-------  -------  -------  --------
1.32292  3.89615  0.44919  0.568958


1_plus_blood eq 1
    OR      UC       LC       p_val
------  ------  -------  ----------
6.8659  23.494  2.00649  0.00049149


10_plus_days eq 1
     OR       UC       LC       p_val
-------  -------  -------  ----------
7.4062

In [114]:
# Logistic Regression with significant Variables
import statsmodels.api as sm


logit = sm.Logit(df['dummy_ssi'].astype('int'), df[sig_vars].astype('int'))

# fit the model
result = logit.fit()
print(result.summary())
print("ADJUSTED Odds Ratios")
params = result.params
conf = result.conf_int()
conf['OR'] = params
conf.columns = ['2.5%', '97.5%', 'OR']
print (np.exp(conf))

Optimization terminated successfully.
         Current function value: 0.585322
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:              dummy_ssi   No. Observations:                  178
Model:                          Logit   Df Residuals:                      173
Method:                           MLE   Df Model:                            4
Date:                Fri, 11 Nov 2016   Pseudo R-squ.:                 -0.1808
Time:                        17:03:23   Log-Likelihood:                -104.19
converged:                       True   LL-Null:                       -88.234
                                        LLR p-value:                     1.000
                    coef    std err          z      P>|z|      [95.0% Conf. Int.]
---------------------------------------------------------------------------------
2_plus_abdays    -0.1284      0.371     -0.346      0.729        -0.855     0.598
hollow_visc      -0.

In [141]:
import numpy as np
from sklearn import linear_model

X = df[sig_vars].astype('int')  # we only take the first two features.
Y = df['dummy_ssi'].astype('int')

h = .02  # step size in the mesh

logreg = linear_model.LogisticRegression(C=1e5)

# we create an instance of Neighbours Classifier and fit the data.
logreg.fit(X, Y)
aodds = np.exp(logreg.coef_)
t = tabulate(aodds, headers=sig_vars)
print("Adjusted Odds Ratio: \n")
print(t)


Adjusted Odds Ratio: 

  2_plus_abdays    hollow_visc    1_plus_blood    10_plus_days      vac
---------------  -------------  --------------  --------------  -------
        1.85047          1.662         3.40675         3.67267  1.68025


### Superficial SSI Results:

In [142]:
# odds ratios and P-Vals
# odds ratios and P-Values
sig_vars = []
for var in X_cols:
    t,p_val = OR(df,True,var,'eq',1)
    if p_val < .15:
        sig_vars.append(var)
    print(t)
    print("\n")
print(sig_vars)

2_plus_abdays eq 1
    OR       UC       LC       p_val
------  -------  -------  ----------
4.4494  11.4932  1.72251  0.00202671


female eq 1
     OR       UC        LC    p_val
-------  -------  --------  -------
2.07353  6.92374  0.620983   0.2634


hollow_visc eq 1
     OR       UC        LC     p_val
-------  -------  --------  --------
1.65116  4.31318  0.632094  0.354944


gunshot eq 1
     OR       UC        LC    p_val
-------  -------  --------  -------
1.30797  6.06438  0.282105        1


3_plus_wounds eq 1
     OR       UC        LC     p_val
-------  -------  --------  --------
1.28017  3.22213  0.508621  0.636015


120_plus_pulse eq 1
      OR       UC        LC    p_val
--------  -------  --------  -------
0.764543  3.54478  0.164897        1


1_plus_blood eq 1
     OR       UC        LC      p_val
-------  -------  --------  ---------
3.32673  11.7886  0.938799  0.0813889


10_plus_days eq 1
     OR       UC       LC       p_val
-------  -------  -------  ----------


In [129]:
# Logistic Regression with significant Variables
#stats model is being wierd! use scikit learn??
import statsmodels.api as sm

Y = df['superficial_ssi'].astype('int')
X = df[sig_vars].astype('int')
#print(dummy_df)

logit = sm.Logit(Y,X)

# fit the model
result = logit.fit()
print(result.summary())
print("ADJUSTED Odds Ratios")
print (np.exp(result.params))

Optimization terminated successfully.
         Current function value: 0.513218
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:        superficial_ssi   No. Observations:                  178
Model:                          Logit   Df Residuals:                      174
Method:                           MLE   Df Model:                            3
Date:                Fri, 11 Nov 2016   Pseudo R-squ.:                 -0.4143
Time:                        17:38:47   Log-Likelihood:                -91.353
converged:                       True   LL-Null:                       -64.592
                                        LLR p-value:                     1.000
                    coef    std err          z      P>|z|      [95.0% Conf. Int.]
---------------------------------------------------------------------------------
2_plus_abdays    -0.1091      0.403     -0.271      0.786        -0.898     0.680
1_plus_blood     -1.

In [143]:
import numpy as np
from sklearn import linear_model

X = df[sig_vars].astype('int')  # we only take the first two features.
Y = df['superficial_ssi'].astype('int')

h = .02  # step size in the mesh

logreg = linear_model.LogisticRegression(C=1e5)

# we create an instance of Neighbours Classifier and fit the data.
logreg.fit(X, Y)

aodds = np.exp(logreg.coef_)
t = tabulate(aodds, headers=sig_vars)
print("Adjusted Odds Ratio: \n")
print(t)

Adjusted Odds Ratio: 

  2_plus_abdays    1_plus_blood    10_plus_days      vac
---------------  --------------  --------------  -------
        3.29472         1.73162         3.70564  1.25289
